In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display

# 지역 이름과 코드 매핑
regions = {
    "남구": "5",
    "부산진구": "8",
    "해운대구": "18",
    "동래구": "7",
    "사하구": "12",
    "북구": "10",
    "사상구": "11"
}

# 날짜 설정
target_date = "2015-07-02"

# 데이터 수집 함수
def fetch_region_data(region_name, region_code):
    url = "https://www.busan.go.kr/depart/abnecessity01"
    params = {
        "isMart": "true",
        "kind": "",
        "schDate": target_date,
        "schCode": "0",
        "schLocale": region_code,
        "schMart": "0"
    }

    response = requests.get(url, params=params)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find("table", class_="boardList")

    region_data = []

    if table:
        rows = table.find_all("tr")
        for row in rows:
            tds = row.find_all("td")
            td_texts = [td.get_text(strip=True) for td in tds]
            if len(td_texts) >= 5:
                item_name = td_texts[0]
                spec = td_texts[1]
                try:
                    lowest_price = int(td_texts[2].replace(",", ""))
                    highest_price = int(td_texts[3].replace(",", ""))
                except ValueError:
                    continue
                avg_price = td_texts[4]
                region_data.append([item_name, spec, lowest_price, highest_price, avg_price, region_name])
    return region_data

# 모든 지역에서 데이터 수집
all_data = []
for region_name, region_code in regions.items():
    all_data.extend(fetch_region_data(region_name, region_code))

# DataFrame 생성
columns = ["품목", "규격", "최저가", "최고가", "금주평균가", "지역"]
df = pd.DataFrame(all_data, columns=columns)

# 최저가, 최고가 각각 추출
min_df = df.loc[df.groupby("품목")["최저가"].idxmin()][["품목", "규격", "최저가", "지역", "금주평균가"]].reset_index(drop=True)
max_df = df.loc[df.groupby("품목")["최고가"].idxmax()][["품목", "최고가", "지역"]].reset_index(drop=True)

# 컬럼명 변경
min_df = min_df.rename(columns={"지역": "최저가_지역"})
max_df = max_df.rename(columns={"지역": "최고가_지역"})

# 병합
merged_df = pd.merge(min_df, max_df, on="품목")

# 가격 차이 계산
merged_df["가격차이"] = merged_df["최고가"] - merged_df["최저가"]
merged_df["차이비율(%)"] = (merged_df["가격차이"] / merged_df["최저가"] * 100).round(2).astype(str) + "%"

# 날짜 열 추가
merged_df["날짜"] = target_date

# 열 정리 및 출력
merged_df = merged_df[["날짜", "품목", "규격", "최저가", "최저가_지역", "최고가", "최고가_지역", "가격차이", "차이비율(%)", "금주평균가"]]

# 표 출력
display(merged_df)

# 저장하고 싶다면
merged_df.to_csv("2015_total_3q.csv", index=False, encoding="utf-8-sig")


,날짜,품목,규격,최저가,최저가_지역,최고가,최고가_지역,가격차이,차이비율(%),금주평균가
0,2015-07-02,가루비누,"세탁용세제(LG테크),가루형,3.0kg",8162,남구,17850,남구,9688,118.7%,"13,006"
1,2015-07-02,간장,오복왕표 0.9ℓ 1병,3755,해운대구,8500,남구,4745,126.36%,"5,518"
2,2015-07-02,고등어,30㎝정도 500g 1마리,2500,북구,6413,북구,3913,156.52%,"4,457"
3,2015-07-02,달걀,오경슈퍼란60g정도 10개,1326,해운대구,3150,부산진구,1824,137.56%,"2,309"
4,2015-07-02,닭고기,육계 1.0㎏,4750,해운대구,13990,부산진구,9240,194.53%,"6,924"
5,2015-07-02,대파,1.0㎏,2700,북구,4700,동래구,2000,74.07%,"3,550"
6,2015-07-02,돼지고기,삼겹살 500g,8900,동래구,14950,사하구,6050,67.98%,"11,400"
7,2015-07-02,두부,500g 판두부(국산포장두부 420g) 1모,2996,남구,5000,남구,2004,66.89%,"3,998"
8,2015-07-02,라면,신라면 120g 1봉지,572,사상구,640,동래구,68,11.89%,603
9,2015-07-02,맥주,하이트 500㎖ 1병,1200,북구,1350,사하구,150,12.5%,"1,230"
